# Feature Engineering:

In [9]:
import pandas as pd
from datetime import timedelta

# Load CSV
df = pd.read_csv("merged_output.csv", parse_dates=['block_timestamp'])

# Filter only for blocks in 2017
df_2017 = df[df['Year'] == 2020].copy()


df_2017.head()

,Block Height,block_difficulty,block_timestamp,Year,Address,Value,Miner Pool Name
332690,610691,1715bcd0,2020-01-01 00:03:05+00:00,2020,1KDf847BWEMwthPHFtvWAfESdQMRnr25f7,1267534026,Poolin
332691,610692,1715bcd0,2020-01-01 00:07:32+00:00,2020,1C81BGyi8SJ919UxnHm2iwNmaN2RHfU4us,1255524017,Huobi.pool
332692,610693,1715bcd0,2020-01-01 00:13:34+00:00,2020,1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY,1254319091,F2Pool
332693,610694,1715bcd0,2020-01-01 00:19:00+00:00,2020,3QLeXx1J9Tp3TBnQyHrhVxne9KqkAS9JSR,1252776912,CN/TT
332694,610695,1715bcd0,2020-01-01 00:37:32+00:00,2020,12dRugNcdxK39288NjcDV4GX7rMsKCGn6B,1258768740,AntPool


## 1. blocks_per_pool_per_day

Number of blocks mined by a specific pool in the 24 hours before the current block.

🔸 Beikverdi & Song (2015):
Highlighted the growth of large mining pools as a centralizing force.

The 24-hour block count reflects recent dominance, helping detect short-term control by large pools.

🔸 Zolotavkin et al. (2019):
Noted the increase in power concentration as a result of consistent pool growth.

This feature captures the daily participation strength of a pool, which aligns with their concern about long-standing, large pools mining the majority of blocks.

🔸 Ren & Ward (2021):
Discussed the increasing share of mining controlled by a few entities.

A pool's daily block count is a direct indicator of its active presence, especially if it exceeds a certain threshold.

In [10]:
# Sort by time for accurate sliding window
df_2017 = df_2017.sort_values('block_timestamp').reset_index(drop=True)

# Initialize the new feature column
df_2017['blocks_per_pool_per_day'] = 0

# Loop through each block
for i in range(len(df_2017)):
    current_time = df_2017.loc[i, 'block_timestamp']
    pool = df_2017.loc[i, 'Miner Pool Name']
    
    # Define the 24-hour window
    start_time = current_time - timedelta(days=1)

    # Filter for blocks from same pool in the last 24h before current block
    mask = (df_2017['block_timestamp'] >= start_time) & (df_2017['block_timestamp'] < current_time)
    recent_blocks = df_2017[mask & (df_2017['Miner Pool Name'] == pool)]
    
    # Count and assign
    df_2017.loc[i, 'blocks_per_pool_per_day'] = recent_blocks.shape[0]

## 2. blocks_per_pool_last_10_blocks

How many of the last 10 blocks were mined by the same pool as the current block.

🔸 Long et al. (2022):
Introduced the “previous block advantage”, showing that miners who mine one block have a higher chance of mining the next due to latency or network advantage.

If a pool appears in several of the last 10 blocks, it could be exhibiting this temporal advantage — a short-term centralizing trend.

🔸 Zolotavkin et al. (2019):
Emphasized short-term concentration risks in mining activity.

This feature helps capture bursts of mining activity, which may signa90l opportunistic control or selfish mining attempts.

🔸 Chen et al. (2023) – “Endex”:
Proposed a temporal-aware decentralization index.

This kind of short-term clustering (in last 10 blocks) supports detecting spikes in centralization, aligning with their time-sensitive modeling approach.



In [11]:
# Initialize the column
df_2017['blocks_per_pool_last_10'] = 0

# Loop over the DataFrame
for i in range(len(df_2017)):
    pool = df_2017.loc[i, 'Miner Pool Name']
    
    # Adjust the range to avoid negative indexing
    start_idx = max(0, i - 10)
    last_blocks = df_2017.iloc[start_idx:i]
    
    # Count how many of those blocks were mined by the same pool
    count = (last_blocks['Miner Pool Name'] == pool).sum()
    df_2017.loc[i, 'blocks_per_pool_last_10'] = count

# 3. is_unique_miner_in last_24_hour

Take value of true if the miner appears first time in a day(e.g., last 100 blocks or per day).

More unique miners → more participation, better decentralization

Fewer → potential monopolization

🔸 Beikverdi & Song (2015):
Raised concern over the shrinking number of active mining entities.

🔸 Ren & Ward (2021):
Show that as the number of effective miners drops, centralization increases.




In [ ]:
# Step 1: Extract the date (if not already done)
df_2017['day'] = df_2017['block_timestamp'].dt.date

# Step 2: Sort by timestamp to ensure correct first appearances
df_2017 = df_2017.sort_values(by=['day', 'block_timestamp']).reset_index(drop=True)

# Step 3: Mark the first block per (day, Miner Pool Name) as unique
df_2017['is_unique_miner'] = df_2017.duplicated(subset=['day', 'Miner Pool Name'], keep='first').apply(lambda x: 0 if x else 1)


# 4. is_unique_miner_in last_10_block

Take value of true if the miner appears first time in a day(e.g., last 100 blocks or per day).

More unique miners → more participation, better decentralization

Fewer → potential monopolization

🔸 Beikverdi & Song (2015):
Raised concern over the shrinking number of active mining entities.

🔸 Ren & Ward (2021):
Show that as the number of effective miners drops, centralization increases.




In [12]:
# Initialize the new column
df_2017['is_unique_miner_last_10'] = 0

# Loop through the dataset
for i in range(len(df_2017)):
    current_pool = df_2017.loc[i, 'Miner Pool Name']
    
    # Define the previous 100-block window
    start_idx = max(0, i - 10)
    window = df_2017.iloc[start_idx:i]
    
    # Check if this miner has appeared in the last 100 blocks
    is_first_in_10 = current_pool not in window['Miner Pool Name'].values
    
    df_2017.loc[i, 'is_unique_miner_last_10'] = int(is_first_in_10)


# 5. Is_consecutive_block

Take true value if it is a consecutive block.

Large streaks may suggest less decentrlization

🔸 Long et al. (2022):
Introduced the concept of previous block advantage, showing how winning one block increases chance of winning the next.

🔸 Zolotavkin et al. (2019):
Discuss the danger of short-term dominance spikes.




In [13]:
# Initialize the column with 0
df_2017['is_consecutive_block'] = 0

# Compare each block with the previous one
df_2017['prev_pool'] = df_2017['Miner Pool Name'].shift(1)

# If same as previous, mark as 1
df_2017.loc[df_2017['Miner Pool Name'] == df_2017['prev_pool'], 'is_consecutive_block'] = 1

# Clean up (optional)
df_2017.drop(columns=['prev_pool'], inplace=True)


In [14]:
df_2017.to_csv('Blocks with feature2020.csv')